# Get 5 min volume for Feb 2018 from Unprocessed Speed

    Created by: Apoorba Bibeka
    Date: March 20, 2019

## Import required modules

In [1]:
import getpass
import boto3
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
from io import BytesIO as StringIO

## Connect to the database

In [2]:
username = "abibeka@securedatacommons.com"
password1 = getpass.getpass()

········


In [3]:
conn = connect(host="172.18.1.20",auth_mechanism='PLAIN',port=10000,user=username, password=password1)

## Look at the device ids from "wydot_speed_sensors_index"

In [4]:
cursor = conn.cursor()
cursor.execute('select * from wydot_speed_sensors_index')
# Store the index data tanle
SSindex = as_pandas(cursor)
# Remove "wydot_speed_sensors_index." from the col names
Rename1 ={x:x.split('wydot_speed_sensors_index.')[1] for x in SSindex}
SSindex=SSindex.rename(index=str,columns=Rename1)
print(SSindex['deviceid'].values,SSindex.columns)

(array([ 382,  383,  393,  394,  395,  396,  398,  400,  405,  407,  408,
        411,  384,  482, 1075, 1084, 1100, 1134, 1145, 1153, 1167, 1219,
       1231,  385, 1241, 1251, 1258, 1269, 1280, 1327, 1342, 1837, 1838,
       1839,  386, 2020, 2032, 2049, 2070, 2079, 2090, 2146, 2147, 2178,
       2191,  387, 2202, 2213, 2246, 2263, 2274, 2289, 2298, 2310, 2319,
       2334,  388, 2346, 2359, 2372, 2383, 2395, 2409, 2421, 2433, 2445,
       2578,  389, 2607, 2609, 2916, 3236, 3243, 3249, 3296, 3402, 3482,
       3654,  390, 3897, 3899, 3901, 3903, 3905, 3907, 3909, 3911,  391], dtype=int64), Index([u'deviceid', u'lat_decimal', u'long_decimal', u'road_code', u'sitename',
       u'devicename', u'sensortype', u'public_route', u'gis_route',
       u'direction', u'milepost', u'sensor_loc', u'nearest_rwis', u'rwis',
       u'backup_rwis', u'2015_adt', u'vsl_id', u'eb_vsl', u'wb_vsl',
       u'horiz_d', u'horiz_i', u'vert_i', u'vert_d', u'notes'],
      dtype='object'))


In [5]:
SSindex

,deviceid,lat_decimal,long_decimal,road_code,sitename,devicename,sensortype,public_route,gis_route,direction,...,backup_rwis,2015_adt,vsl_id,eb_vsl,wb_vsl,horiz_d,horiz_i,vert_i,vert_d,notes
0,382,41.172596,-104.083160,,Pine Bluffs,Speed Sensor,Traffic,I80,ML80B,B,...,R003451,8147,N5,1,2,0,0,-0.54,0.37,
1,383,41.213795,-105.440920,,Summit East,Speed Sensor,Traffic,I80/US30,ML80B,B,...,R000360,13890,V4,2136,2148,0,0,0.60,-0.06,No data available
2,393,41.151546,-105.393330,,Vedauwoo,Speed Sensor,Traffic,I80/US30,ML80B,I,...,R003422,13890,V4,2170,2179,0,0,-3.12,2.80,No data available
3,394,41.226612,-105.438740,,Summit East,Speed Sensor,Traffic,I80/US30,ML80B,D,...,R000360,13890,V4,2136,2148,0,0,0.32,-0.38,No data available
4,395,41.258514,-105.453320,,Summit,Speed Sensor,Traffic,I80/US30,ML80B,B,...,R000360,13890,V4,2118,2128,0,4,3.75,-3.75,No data available
5,396,41.279910,-105.516990,,Laramie East,Speed Sensor,Traffic,I80/US30,ML80B,B,...,R000360,13890,V4,2118,3668,0,0,1.83,-3.24,
6,398,41.657383,-106.347050,,CR 402 East,Speed Sensor,Traffic,I80,ML80B,I,...,R001242,10645,V3,433,430,0,0,0.50,-0.62,
7,400,41.693750,-106.384050,,CR 402 West,Speed Sensor,Traffic,I80,ML80B,B,...,R001220,10645,V3,438,430,0,0,1.63,-1.97,
8,405,41.513710,-106.054560,,Cooper Cove East,Speed Sensor,Traffic,I80,ML80B,B,...,R001281,10645,V3,417,416,0,0,2.14,-2.03,
9,407,41.741460,-106.839290,,Walcott Junction,Speed Sensor,Traffic,I80/US287/US30,ML80B,I,...,R001191,10645,N3,1,2,0,0,-0.37,0.96,


### Subset data for Lamarie & Cheyenne

In [6]:
SS2index=SSindex.loc[(SSindex['milepost']>=314)&(SSindex['milepost']<=360) & (SSindex['direction']!='B')]
SS2index = SS2index[["deviceid","milepost","direction","2015_adt"]].copy()
print(SS2index)

    deviceid  milepost direction  2015_adt
2        393    330.00         I     13890
3        394    324.00         D     13890
12       384    325.80         I     13890
44      2191    332.29         D     13890
46      2202    333.32         I     13890
47      2213    334.30         D     13890
77      3654    318.50         D     13890


## Get the index Data (314 <= Milepost <=360) from S3 bucket (Source: WyDoT Report)

In [7]:
'''#Data I got from a Report
client=boto3.client('s3')
# Get the data from our bucket directly
obj=client.get_object(Bucket='prod-sdc-tti-911061262852-us-east-1-bucket',Key="abibeka/Data/SpeedSen_MP_314_360.csv")
AB_SS=pd.read_csv(obj['Body'])
AB_SS.head()'''

'#Data I got from a Report\nclient=boto3.client(\'s3\')\n# Get the data from our bucket directly\nobj=client.get_object(Bucket=\'prod-sdc-tti-911061262852-us-east-1-bucket\',Key="abibeka/Data/SpeedSen_MP_314_360.csv")\nAB_SS=pd.read_csv(obj[\'Body\'])\nAB_SS.head()'

## Compare index from WyDOT Report and Database

In [8]:
'''# Return set with elements in either the set or ohter but not both
print(set(AB_SS['DEVICEID'].values) ^ set(SS2index['deviceid'].values))
# Return intersection 
print(set(AB_SS['DEVICEID'].values) & set(SS2index['deviceid'].values))'''

"# Return set with elements in either the set or ohter but not both\nprint(set(AB_SS['DEVICEID'].values) ^ set(SS2index['deviceid'].values))\n# Return intersection \nprint(set(AB_SS['DEVICEID'].values) & set(SS2index['deviceid'].values))"

## Get relevant columns and Rows from the Index Data

In [9]:
'''SS2index=SS2index.sort_values(by="milepost")
SS2index=SS2index.reset_index(drop=True)
SS2index.head()'''

'SS2index=SS2index.sort_values(by="milepost")\nSS2index=SS2index.reset_index(drop=True)\nSS2index.head()'

## Initial Approach to get Volume Data (Not good - Can Aggregate Directly on Hive)

In [10]:
#create_query='''SELECT t1.controller, t1.mountain, t2.direction, t2.milepost, t2.2015_adt FROM 
#(SELECT * FROM wydot_speed_unprocessed WHERE mountain between '2018-02-01' AND '2018-02-28') t1
#JOIN wydot_speed_sensors_index t2 
#ON (t1.controller = t2.deviceid)'''

#create_query='''SELECT t1.controller, t1.mountain, t2.direction, t2.milepost, t2.2015_adt FROM 
#(SELECT * FROM wydot_speed_unprocessed WHERE mountain between '2018-02-01' AND '2018-02-10') t1
#JOIN wydot_speed_sensors_index t2 
#ON (t1.controller = t2.deviceid)'''
#cursor.execute(create_query)
#Vol_dat=as_pandas(cursor)
#Vol_dat.to_csv("C:/Users/abibeka/Documents/Hive-SDC/Vol_dat.csv")
#print(cursor.fetchall())

## $2^{nd}$ Approach to get Count Data

### Specify Start and End Date for Data Aggregation

In [11]:
# Store the data to Z drive
#Vol_dat2.to_csv("Z:/Apoorb/Vol_dat_V1.csv")

# Get volume and Speed data divided by lane and vehicle type

In [13]:
start_date='2018-02-01'
end_date='2018-03-01' 
create_query='''SELECT t1.controller, t1.lane, t1.vehclass, t1.Time5M, t1.speedmph_avg, t1.NRec, t2.direction, t2.milepost, t2.2015_adt FROM 
    (SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300) AS Time5M, lane, vehclass, AVG(speedmph) AS speedmph_avg, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}'
    GROUP BY controller, lane, vehclass,FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300)) t1
JOIN (SELECT * FROM wydot_speed_sensors_index WHERE milepost BETWEEN 314 AND 360) t2 
ON (t1.controller = t2.deviceid)
ORDER BY t1.controller, t1.Time5M, t1.lane, t1.vehclass'''.format(start_date,end_date)
cursor.execute(create_query)
Vol_dat3=as_pandas(cursor)
#direction != 'B' AND

In [14]:
Vol_dat3.head(100)

,t1.controller,t1.lane,t1.vehclass,t1.time5m,t1.speedmph_avg,t1.nrec,t2.direction,t2.milepost,t2.2015_adt
0,384,1,1,2018-02-01 00:05:00,NaN,1,I,325.8,13890
1,384,1,2,2018-02-01 00:05:00,NaN,1,I,325.8,13890
2,384,1,3,2018-02-01 00:05:00,63.550001,4,I,325.8,13890
3,384,3,3,2018-02-01 00:05:00,65.703334,3,I,325.8,13890
4,384,4,1,2018-02-01 00:05:00,65.650002,1,I,325.8,13890
5,384,4,2,2018-02-01 00:05:00,60.340000,1,I,325.8,13890
6,384,4,3,2018-02-01 00:05:00,58.105556,9,I,325.8,13890
7,384,1,3,2018-02-01 00:10:00,59.123333,6,I,325.8,13890
8,384,4,2,2018-02-01 00:10:00,63.280001,2,I,325.8,13890
9,384,4,3,2018-02-01 00:10:00,62.209999,1,I,325.8,13890


In [ ]:
start_date='2018-02-01'
end_date='2018-03-01' 
q1='''SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300) AS Time5M, lane, vehclass, AVG(speedmph) AS speedmph_avg, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}' AND controller = 384
    GROUP BY controller, lane, vehclass,FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300)'''.format(start_date,end_date)
cursor.execute(q1)
volDebug=as_pandas(cursor)

In [ ]:
volDebug.head(100)

In [ ]:
import numpy as np
np.sum(np.isnan(Vol_dat3['t1.speedmph_avg']))

# Save aggregate data by lane and vehicle class

In [15]:
csv_buffer=StringIO()
Vol_dat3.to_csv(csv_buffer,sep=",",index=False)
s3=boto3.resource('s3')
s3.Object('prod-sdc-tti-911061262852-us-east-1-bucket','abibeka/Data/Feb2018HiveVolDat_Ln_Cls.csv').put(Body=csv_buffer.getvalue())

{u'ETag': '"45f0d03a1c090b250c93b30f1ff29672"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Thu, 25 Apr 2019 19:20:36 GMT',
   'etag': '"45f0d03a1c090b250c93b30f1ff29672"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'tiKToJJmXpBlNZ2sH2QK4jN2EpuBYYAeIIcykfv4/9vkgafNSckJ3+ca59VyWnEW3FiLyMgP1ko=',
   'x-amz-request-id': 'DF5C1744D23971A6'},
  'HTTPStatusCode': 200,
  'HostId': 'tiKToJJmXpBlNZ2sH2QK4jN2EpuBYYAeIIcykfv4/9vkgafNSckJ3+ca59VyWnEW3FiLyMgP1ko=',
  'RequestId': 'DF5C1744D23971A6',
  'RetryAttempts': 0}}